Getting started

In [1]:
import spotify_enrichment

In [3]:
song_measures, song_location = spotify_enrichment.spotify_main()
print(song_measures)

Getting song info for california
dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])


TypeError: cannot unpack non-iterable NoneType object